# Practica uno

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica solo se usará la librería de numpy.

In [ ]:
import numpy as np

## Ejercicio 1



In [ ]:
class CodificadorEtiqueta:
	def __init__(self):
		self.codificacion = None
		self.decodificacion = None

	def inicializar(self, y: list):
		if not isinstance(y, list):
			raise TypeError("La entrada debe ser una lista")

		valores_unicos = np.unique(y) #Conjunto de valores únicos
		self.codificacion={valor: i for i, valor in enumerate(valores_unicos)} #Diccionario de codificación
		self.decodificacion={i: valor for i, valor in enumerate(valores_unicos)} #Diccionario de decodificación

	def codificar(self, y: list):
		try:
			return [self.codificacion[valor] for valor in y] #Codifica los valores de y
		except KeyError as e:
			raise KeyError('Etiqueta no reconocida: ', e)

	def decodificar(self, y: list):
		try:
			return [self.decodificacion[valor] for valor in y] #Decodifica los valores de y
		except KeyError as e:
			raise KeyError('Valor no tiene ninguna etiqueta asociada: ', e)

In [ ]:
codificador = CodificadorEtiqueta() #Instancia de la clase CodificadorEtiqueta
codificador.inicializar(['a', 'b', 'c', 'a'])

In [ ]:
codificador.codificar(['a', 'b', 'c', 'a'])
codificador.decodificar([0, 1, 2, 0, 2, 2, 2, 1, 0, 0])
try:
    codificador.codificar("juan")
except KeyError as e:
    print(e)
try:
    codificador.decodificar([7,7,7,7])
except KeyError as e:
    print(e)

## Ejercicio 2

In [ ]:
class EscalarValores:
	def __init__(self):
		self.max = None
		self.min = None
		self.x_max = None
		self.x_min = None
	
	def inicializar(self, x: np.ndarray, min: float=-1.0, max: float=1) -> None:
		"""
		Inicializa el escalador con los valores máximos y mínimos de los datos

		Para el máximo y mínimo, basta con guardar un número para ambos.
		Pero para los valores máximos y mínimos de los datos, estos se guardan como vectores fila,
		donde cada elemento del vector es el máximo o mínimo de cada columna de los datos.
		"""
		self.max = max
		self.min = min
		self.x_max = np.max(x, axis=0)
		self.x_min = np.min(x, axis=0)

	def escalar(self, x: np.ndarray) -> np.ndarray:
		if (self.x_max is None or self.x_min is None or self.max is None or self.min is None):
			raise Exception("No se ha inicializado el escalador")
		data_maxs = np.max(x, axis=0)
		data_mins = np.min(x, axis=0)
		if (np.any(data_maxs > self.x_max) or np.any(data_mins < self.x_min)):
			raise Exception("Valores fuera de rango")
		max, min, x_max, x_min = self.max, self.min, self.x_max, self.x_min
		return (max -min) * x / (x_max - x_min) + (min * x_max -max * x_min) / (x_max - x_min)

	def escalar_inv(self, x: np.ndarray) -> np.ndarray:
		if (self.x_max is None or self.x_min is None or self.max is None or self.min is None):
			raise Exception("No se ha inicializado el escalador")
		if(np.any(x > self.max) or np.any(x < self.min)):
			raise Exception("Valores fuera de rango")
		max, min, x_max, x_min = self.max, self.min, self.x_max, self.x_min
		return (x_max - x_min) * x / (max - min) + (x_min * max - x_max * min) / (max - min)

In [ ]:
xs = np.array([
	[0, 10],
	[1, 20]
])

m = EscalarValores()

try:
	m.escalar(xs)
except Exception as e:
	print(e)

m.inicializar(xs)
print(m.escalar(xs))
print(m.escalar(np.array([[0.5, 15], [0.7, 12]])))
print(m.escalar_inv(m.escalar(xs)))

try:
	m.escalar(np.array([[1.2, 21]]))
except Exception as e:
	print(e)

## Ejercicio 3

In [ ]:
def divide_entrenamiento_test(
	x: np.ndarray,
	y: np.ndarray | None=None,
	tam_train: float=0.7,
	semilla: int | None=None,
	mezclar: bool=True
) -> list:
	total_size = x.shape[0]
	if (y is not None and total_size != y.shape[0]):
		raise Exception("Las dimensiones de x e y no coinciden")
	if (tam_train <= 0 or tam_train >= 1):
		raise Exception("El tamaño de entrenamiento debe estar entre 0 y 1")
	has_y = y is not None
	train_size = int(total_size * tam_train)
	transform_mat = x.copy()
	if(has_y):
		"""
		Se concatena la columna de y a la matriz de datos.
		De esta forma, se pueden mezclar los datos y las etiquetas sin perder la correspondencia.
		"""
		transform_mat = np.hstack((x, np.array([y]).T))
	if(mezclar):
		np.random.seed(semilla)
		np.random.shuffle(transform_mat)
	
	train = transform_mat[:train_size]
	test = transform_mat[train_size:]
	if(has_y):
		return [train[:, :-1], test[:, :-1], train[:, -1], test[:, -1]]
	else:
		return [train, test]


In [ ]:
x = np.array([
  [1, 2, 3],
  [4, 5, 6],
  [7, 8, 9],
  [10, 11, 12],
  [13, 14, 15],
  [16, 17, 18],
  [19, 20, 21],
])

y = np.array([10, 20, 30, 40, 50, 60, 70])

xa, xb, ya, yb = divide_entrenamiento_test(x, y, 0.5, mezclar=True, semilla=42)

print(xa)
print()
print(ya)
print()
print(xb)
print()
print(yb)